# Required Packages

In [2]:
options(warn=-1)
library(data.table)
library(ggplot2)
library(rpart)
library(wavelets)
library(TSrepr)
library(TSdist)
library(dtw)
library(stringdist)

# Data Manipulations

In [3]:
dt1 <- fread("bulk_imbalance.csv")
dt1 <- dt1[, `:=`(date=as.Date(date, format="%Y-%m-%d"),
                  hour=as.numeric(hour),
                  datetime=as.POSIXct(paste(date, hour), format="%Y-%m-%d %H"))]
dt1 <- dt1[order(datetime)]
head(dt1)

date,hour,net,upRegulationZeroCoded,upRegulationOneCoded,upRegulationTwoCoded,downRegulationZeroCoded,downRegulationOneCoded,downRegulationTwoCoded,upRegulationDelivered,downRegulationDelivered,system_direction,datetime
2019-01-01,0,-1934.795,0,0,0,2288.567,0,0,0,-1934.795,Negative,2019-01-01 00:00:00
2019-01-01,1,-750.866,0,0,0,808.083,0,0,0,-750.866,Negative,2019-01-01 01:00:00
2019-01-01,2,-982.767,0,0,0,1060.750,0,0,0,-982.767,Negative,2019-01-01 02:00:00
2019-01-01,3,-764.620,0,0,0,1120.483,0,0,0,-764.620,Negative,2019-01-01 03:00:00
2019-01-01,4,-1233.289,0,0,0,1874.633,56,0,0,-1233.289,Negative,2019-01-01 04:00:00
2019-01-01,5,-1469.061,0,0,0,2125.000,56,0,0,-1469.061,Negative,2019-01-01 05:00:00


In [4]:
dt2 <- fread("2022-01-19_weather.csv")
dt2 <- dt2[, `:=`(date=as.Date(date, format="%Y-%m-%d"),
                  hour=as.numeric(hour),
                  datetime=as.POSIXct(paste(date, hour), format="%Y-%m-%d %H"))]
dt2 <- dt2[order(datetime)]

dt2 <- dcast(data=dt2, datetime+lat+lon~variable, fun.aggregate=mean, value.var="value")
dt2 <- dt2[, list(meanlat=mean(lat), meanlon=mean(lon), meanRH_2.m.above.ground=mean(RH_2.m.above.ground), 
                  meanTCDC_low.cloud.layer=mean(TCDC_low.cloud.layer), meanTMP_2.m.above.ground=mean(TMP_2.m.above.ground),
                  meanwdir_10m=mean(wdir_10m), meanws_10m=mean(ws_10m)), by=datetime]
head(dt2)

datetime,meanlat,meanlon,meanRH_2.m.above.ground,meanTCDC_low.cloud.layer,meanTMP_2.m.above.ground,meanwdir_10m,meanws_10m
2018-12-31 00:00:00,38.67857,31.39286,83.84286,36.71429,-0.9582857,228.9641,1.924707
2018-12-31 01:00:00,38.67857,31.39286,83.39048,34.19048,-1.1132381,233.0768,1.910163
2018-12-31 02:00:00,38.67857,31.39286,82.93810,31.66667,-1.2681905,237.1896,1.895618
2018-12-31 03:00:00,38.67857,31.39286,82.48571,29.14286,-1.4231429,241.3023,1.881074
2018-12-31 04:00:00,38.67857,31.39286,82.04762,22.71429,-1.6700952,234.1067,1.858578
2018-12-31 05:00:00,38.67857,31.39286,81.60952,16.28571,-1.9170476,226.9110,1.836082


In [5]:
dt <- merge(dt1, dt2, by="datetime")
head(dt)

datetime,date,hour,net,upRegulationZeroCoded,upRegulationOneCoded,upRegulationTwoCoded,downRegulationZeroCoded,downRegulationOneCoded,downRegulationTwoCoded,upRegulationDelivered,downRegulationDelivered,system_direction,meanlat,meanlon,meanRH_2.m.above.ground,meanTCDC_low.cloud.layer,meanTMP_2.m.above.ground,meanwdir_10m,meanws_10m
2019-01-01 00:00:00,2019-01-01,0,-1934.795,0,0,0,2288.567,0,0,0,-1934.795,Negative,38.67857,31.39286,75.25714,4.142857,-0.5152857,266.6930,3.402528
2019-01-01 01:00:00,2019-01-01,1,-750.866,0,0,0,808.083,0,0,0,-750.866,Negative,38.67857,31.39286,75.27143,5.476190,-0.4627143,265.3387,3.649676
2019-01-01 02:00:00,2019-01-01,2,-982.767,0,0,0,1060.750,0,0,0,-982.767,Negative,38.67857,31.39286,75.28571,6.809524,-0.4101429,263.9843,3.896824
2019-01-01 03:00:00,2019-01-01,3,-764.620,0,0,0,1120.483,0,0,0,-764.620,Negative,38.67857,31.39286,75.30000,8.142857,-0.3575714,262.6300,4.143971
2019-01-01 04:00:00,2019-01-01,4,-1233.289,0,0,0,1874.633,56,0,0,-1233.289,Negative,38.67857,31.39286,76.42381,14.809524,-0.2638571,260.4213,4.249762
2019-01-01 05:00:00,2019-01-01,5,-1469.061,0,0,0,2125.000,56,0,0,-1469.061,Negative,38.67857,31.39286,77.54762,21.476190,-0.1701429,258.2125,4.355552


In [6]:
write.csv(dt, "dt.csv", row.names = TRUE)

# Function Definitions

In [7]:
getDataTableReady <- function(givenTable, predictedHour, windowSize) {
    myTable <- data.table(datetime=as.POSIXct(NA), date=as.Date(NA), hour=NA, net=NA, upRegulationZeroCoded=NA, 
                         upRegulationOneCoded=NA, upRegulationTwoCoded=NA, downRegulationZeroCoded=NA, 
                         downRegulationOneCoded=NA, downRegulationTwoCoded=NA, upRegulationDelivered=NA,
                         downRegulationDelivered=NA, system_direction=NA, meanlat=NA, meanlon=NA, 
                         meanRH_2.m.above.ground=NA, meanTCDC_low.cloud.layer=NA, meanTMP_2.m.above.ground=NA,
                         meanwdir_10m=NA, meanws_10m=NA)
    
    #if the data is downloaded at 11am, the last available raw data is at 5am
    buffer_interval <- predictedHour-5

    for (i in which(givenTable$hour==predictedHour)) {
        #if the first time serie does not have enough observation(window size)
        if(i-windowSize-buffer_interval<=0) {
            next          
            }
        else {
            tempTable <- rbind(givenTable[(i-windowSize-buffer_interval+1):(i-buffer_interval)], givenTable[i])
            myTable <- rbind(myTable, tempTable) 
            }
        }
    
    myTable <- myTable[-1]
    myTable[, id:=rep(1:(nrow(myTable)/(windowSize+1)), each=(windowSize+1))]
    myTable[system_direction=="" & net>=50, system_direction:="Positive"]
    myTable[system_direction=="" & net<50 & net>-50, system_direction:="Neutral"]
    myTable[system_direction=="" & net<=-50, system_direction:="Negative"]
    return(myTable)
}

In [8]:
actualClassesFinder <- function(givenTable) {    
    maxID <- as.numeric(givenTable[which.max(givenTable$id)]$id)
    windowSize <- (nrow(givenTable)/maxID)-1
    targetIndices <- seq(from=(windowSize+1), to=nrow(givenTable), by=(windowSize+1))
    return(givenTable[targetIndices]$system_direction)
}

In [9]:
rawReprFunc <- function(givenTable) {
    myTable <- copy(givenTable)
    maxID <- as.numeric(myTable[which.max(myTable$id)]$id)
    windowSize <- (nrow(myTable)/maxID)-1   
    reprValuesVec <- numeric(0)
    
    for (i in (1:maxID)) {
        reprValuesVec <- c(reprValuesVec, myTable[id==i][-(windowSize+1)]$net)      
    }
    
    myMatrix <- matrix(reprValuesVec, nrow=maxID, ncol=windowSize, byrow=T)
    return(myMatrix) 
}

In [10]:
treeReprFunc <- function(givenTable, maxDepth) {
    myTable <- copy(givenTable)
    maxID <- as.numeric(myTable[which.max(myTable$id)]$id)
    windowSize <- (nrow(myTable)/maxID)-1   
    reprValuesVec <- numeric(0)

    
    for (i in (1:maxID)) {
        
        treeFit <- rpart(net~datetime, myTable[id==i][1:windowSize],
                         control=rpart.control(maxdepth=maxDepth, cp=0, minsplit=3, minbucket=1))
        reprValuesVec <- c(reprValuesVec, predict(treeFit, myTable[id==i][1:windowSize]))
    }
   
    myMatrix <- matrix(reprValuesVec, nrow=maxID, ncol=windowSize, byrow=T)
    return(myMatrix)
}

In [11]:
waveletReprFunc <- function(givenTable, givenMethod) {
    myTable <- copy(givenTable)
    maxID <- as.numeric(myTable[which.max(myTable$id)]$id)
    windowSize <- (nrow(myTable)/maxID)-1
    reprValuesVec <- numeric(0)

    for (i in (1:maxID)) {
        dwtObject <- dwt(myTable[id==i]$net[1:windowSize], filter=givenMethod)   
        tempValues <- dwtObject@V[[1]] 
        reprValuesVec <- c(reprValuesVec, tempValues)    
    }
    
    myMatrix <- matrix(reprValuesVec, nrow=maxID, ncol=windowSize/2, byrow=T)
    return(myMatrix)
}

In [12]:
saxReprFunc <- function(givenTable, sLength, aSize) {
    myTable <- copy(givenTable)
    maxID <- as.numeric(myTable[which.max(myTable$id)]$id)
    windowSize <- (nrow(myTable)/maxID)-1
    reprValuesVec <- character(0)
    
    for (i in (1:maxID)) {
        SAXRepr <- repr_sax(myTable[id==i]$net[1:windowSize], q=sLength, a=aSize)
        SAXRepr <- paste(SAXRepr, collapse="")
        reprValuesVec <- c(reprValuesVec, SAXRepr)
    }

    return(reprValuesVec)
}

In [13]:
eucDistMatrix <- function(givenTable, predictedTable) {
    eucDist <- as.matrix(TSDatabaseDistances(X=predictedTable, Y=givenTable, distance='euclidean'))
    return(eucDist)
}

manhattanDistMatrix <- function(givenTable, predictedTable) { 
    manhDist <- as.matrix(TSDatabaseDistances(X=predictedTable, Y=givenTable, distance='manhattan'))
    return(manhDist)
}

dtwDistMatrix <- function(givenTable, predictedTable) {
    dtwDist <- as.matrix(dtwDist(mx=predictedTable, my=givenTable))
    return(dtwDist)
}

erpDistMatrix <- function(givenTable, predictedTable) {
    erpDist <- as.matrix(TSDatabaseDistances(X=predictedTable, Y=givenTable, distance='erp', g=0.05))
    return(erpDist)
}

lcssDistMatrixNumeric <- function(givenTable, predictedTable) {
    lcssDist <- as.matrix(TSDatabaseDistances(X=predictedTable, Y=givenTable, distance='lcss', e=0.05))
    return(lcssDist)
}

lcssDistMatrixString <- function(givenTable, predictedTable) {
    lcssDist <- as.matrix(stringdistmatrix(a=predictedTable, b=givenTable, method = "lcs"))
    return(lcssDist)
}

In [14]:
getmode <- function(v) {
    uniqv <- unique(v)
    uniqv[which.max(tabulate(match(v, uniqv)))]
}

kNNCalculator <- function(trainClassVector, distMatrix, k) {    
    KNN_neighborhood <- apply(distMatrix, 1, order)
    tempSplit <- split(trainClassVector[KNN_neighborhood[1:k,]], ceiling(seq_along(trainClassVector[KNN_neighborhood[1:k,]])/k))
    KNN_pred <- sapply(tempSplit, getmode)
    return(KNN_pred)
}

In [15]:
accRowCalculator <- function(actualClasses, predClasses, reprName, distName=NA, wSize, kValue=NA,
                             dMax=NA, method=NA, sLength=NA, aSize=NA) {
    acc <- sum(actualClasses==predClasses)/length(actualClasses)
    return(data.table(repr=reprName, dist=distName, wSize=wSize, kNNValue=kValue, 
                      maxDepth=dMax, method=method, segmentLength=sLength, alphabetSize=aSize, acc=acc))
}

In [16]:
modelComparisonFunc <- function(givenTable, predictedTable, kVector, windowSize) {    
    trainTable <- copy(givenTable)
    testTable <- copy(predictedTable)
    trainingActualClasses <- actualClassesFinder(trainTable)
    testActualClasses <- actualClassesFinder(testTable)
    solutionTable <- data.table(repr=NA, dist=NA, wSize=NA, kNNValue=NA, 
                      maxDepth=NA, method=NA, segmentLength=NA, alphabetSize=NA, acc=NA)
    
    #Raw Representation & Distance Calculations
    rawRepr_train <- rawReprFunc(trainTable)
    rawRepr_test <- rawReprFunc(testTable)
    rawEucMatrix <- eucDistMatrix(rawRepr_train, rawRepr_test)
    rawManhMatrix <- manhattanDistMatrix(rawRepr_train, rawRepr_test)
    rawDtwMatrix <- dtwDistMatrix(rawRepr_train, rawRepr_test)
    rawErpMatrix <- erpDistMatrix(rawRepr_train, rawRepr_test)
    rawLcssMatrix <- lcssDistMatrixNumeric(rawRepr_train, rawRepr_test)   

    #Regression Tree Representation with maxDepth=2 & Distance Calculations    
    treeRepr1_train <- treeReprFunc(trainTable, maxDepth=2)
    treeRepr1_test <- treeReprFunc(testTable, maxDepth=2)
    tree1EucMatrix <- eucDistMatrix(treeRepr1_train, treeRepr1_test)
    tree1ManhMatrix <- manhattanDistMatrix(treeRepr1_train, treeRepr1_test)
    tree1DtwMatrix <- dtwDistMatrix(treeRepr1_train, treeRepr1_test)
    tree1ErpMatrix <- erpDistMatrix(treeRepr1_train, treeRepr1_test)
    tree1LcssMatrix <- lcssDistMatrixNumeric(treeRepr1_train, treeRepr1_test)

    #Regression Tree Representation with maxDepth=6 & Distance Calculations
    treeRepr2_train <- treeReprFunc(trainTable, maxDepth=6)
    treeRepr2_test <- treeReprFunc(testTable, maxDepth=6)
    tree2EucMatrix <- eucDistMatrix(treeRepr2_train, treeRepr2_test)
    tree2ManhMatrix <- manhattanDistMatrix(treeRepr2_train, treeRepr2_test)
    tree2DtwMatrix <- dtwDistMatrix(treeRepr2_train, treeRepr2_test)
    tree2ErpMatrix <- erpDistMatrix(treeRepr2_train, treeRepr2_test)
    tree2LcssMatrix <- lcssDistMatrixNumeric(treeRepr2_train, treeRepr2_test)

    #Discrete Wavelet Transform Representation with method="haar" & Distance Calculations
    dwtRepr1_train <- waveletReprFunc(trainTable, "haar")
    dwtRepr1_test <- waveletReprFunc(testTable, "haar")
    dwt1EucMatrix <- eucDistMatrix(dwtRepr1_train, dwtRepr1_test)
    dwt1ManhMatrix <- manhattanDistMatrix(dwtRepr1_train, dwtRepr1_test)
    dwt1DtwMatrix <- dtwDistMatrix(dwtRepr1_train, dwtRepr1_test)
    dwt1ErpMatrix <- erpDistMatrix(dwtRepr1_train, dwtRepr1_test)
    dwt1LcssMatrix <- lcssDistMatrixNumeric(dwtRepr1_train, dwtRepr1_test)

    #Discrete Wavelet Transform Representation with method="la8" & Distance Calculations
    dwtRepr2_train <- waveletReprFunc(trainTable, "la8")
    dwtRepr2_test <- waveletReprFunc(testTable, "la8")
    dwt2EucMatrix <- eucDistMatrix(dwtRepr2_train, dwtRepr2_test)
    dwt2ManhMatrix <- manhattanDistMatrix(dwtRepr2_train, dwtRepr2_test)
    dwt2DtwMatrix <- dtwDistMatrix(dwtRepr2_train, dwtRepr2_test)
    dwt2ErpMatrix <- erpDistMatrix(dwtRepr2_train, dwtRepr2_test)
    dwt2LcssMatrix <- lcssDistMatrixNumeric(dwtRepr2_train, dwtRepr2_test)
    
    #SAX Representation with sLength=2 and aSize=5 & Distance Calculations
    saxRepr1_train <- saxReprFunc(trainTable, sLength=2, aSize=5)
    saxRepr1_test <- saxReprFunc(testTable, sLength=2, aSize=5)
    sax1LcssMatrix <- lcssDistMatrixString(saxRepr1_train, saxRepr1_test)  
    
    #SAX Representation with sLength=4 and aSize=8 & Distance Calculations
    saxRepr2_train <- saxReprFunc(trainTable, sLength=4, aSize=8)
    saxRepr2_test <- saxReprFunc(testTable, sLength=4, aSize=8)
    sax2LcssMatrix <- lcssDistMatrixString(saxRepr2_train, saxRepr2_test)    
    
    for (k in kVector) {
        ## FOR RAW REPRESENTATION
        #euclidan distance
        predClasses <- kNNCalculator(trainingActualClasses, rawEucMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "raw", "euc", wSize=windowSize, kValue=k,
                                    dMax=NA, method=NA, sLength=NA, aSize=NA)     
        solutionTable <- rbind(solutionTable, tempRow)
        #manhattan distance
        predClasses <- kNNCalculator(trainingActualClasses, rawManhMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "raw", "manh", wSize=windowSize, kValue=k,
                                    dMax=NA, method=NA, sLength=NA, aSize=NA)
        solutionTable <- rbind(solutionTable, tempRow)
        #dtw distance
        predClasses <- kNNCalculator(trainingActualClasses, rawDtwMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "raw", "dtw", wSize=windowSize, kValue=k,
                                    dMax=NA, method=NA, sLength=NA, aSize=NA)
        solutionTable <- rbind(solutionTable, tempRow)
        #erp distance
        predClasses <- kNNCalculator(trainingActualClasses, rawErpMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "raw", "erp", wSize=windowSize, kValue=k,
                                    dMax=NA, method=NA, sLength=NA, aSize=NA)
        solutionTable <- rbind(solutionTable, tempRow)
        #lcss distance
        predClasses <- kNNCalculator(trainingActualClasses, rawLcssMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "raw", "lcss", wSize=windowSize, kValue=k,
                                    dMax=NA, method=NA, sLength=NA, aSize=NA)
        solutionTable <- rbind(solutionTable, tempRow)
        
        ## FOR TREE 1 REPRESENNTATION
        #euclidean distance
        predClasses <- kNNCalculator(trainingActualClasses, tree1EucMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "tree", "euc", wSize=windowSize, kValue=k,
                                    dMax=2, method=NA, sLength=NA, aSize=NA)
        solutionTable <- rbind(solutionTable, tempRow)
        #manhattan distance
        predClasses <- kNNCalculator(trainingActualClasses, tree1ManhMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "tree", "manh", wSize=windowSize, kValue=k,
                                    dMax=2, method=NA, sLength=NA, aSize=NA)
        solutionTable <- rbind(solutionTable, tempRow)
        #dtw distance
        predClasses <- kNNCalculator(trainingActualClasses, tree1DtwMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "tree", "dtw", wSize=windowSize, kValue=k,
                                    dMax=2, method=NA, sLength=NA, aSize=NA)
        solutionTable <- rbind(solutionTable, tempRow)
        #erp distance
        predClasses <- kNNCalculator(trainingActualClasses, tree1ErpMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "tree", "erp", wSize=windowSize, kValue=k,
                                    dMax=2, method=NA, sLength=NA, aSize=NA)
        solutionTable <- rbind(solutionTable, tempRow)
        #lcss distance
        predClasses <- kNNCalculator(trainingActualClasses, tree1LcssMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "tree", "lcss", wSize=windowSize, kValue=k,
                                    dMax=2, method=NA, sLength=NA, aSize=NA)
        solutionTable <- rbind(solutionTable, tempRow)
        
        ## FOR TREE 2 REPRESENTATION
        #euclidean distance
        predClasses <- kNNCalculator(trainingActualClasses, tree2EucMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "tree", "euc", wSize=windowSize, kValue=k,
                                    dMax=6, method=NA, sLength=NA, aSize=NA)
        solutionTable <- rbind(solutionTable, tempRow)
        #manhattan distance
        predClasses <- kNNCalculator(trainingActualClasses, tree2ManhMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "tree", "manh", wSize=windowSize, kValue=k,
                                    dMax=6, method=NA, sLength=NA, aSize=NA)
        solutionTable <- rbind(solutionTable, tempRow)
        #dtw distance
        predClasses <- kNNCalculator(trainingActualClasses, tree2DtwMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "tree", "dtw", wSize=windowSize, kValue=k,
                                    dMax=6, method=NA, sLength=NA, aSize=NA)
        solutionTable <- rbind(solutionTable, tempRow)
        #erp distance
        predClasses <- kNNCalculator(trainingActualClasses, tree2ErpMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "tree", "erp", wSize=windowSize, kValue=k,
                                    dMax=6, method=NA, sLength=NA, aSize=NA)
        solutionTable <- rbind(solutionTable, tempRow)
        #lcss distance
        predClasses <- kNNCalculator(trainingActualClasses, tree2LcssMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "tree", "lcss", wSize=windowSize, kValue=k,
                                    dMax=6, method=NA, sLength=NA, aSize=NA)
        solutionTable <- rbind(solutionTable, tempRow)
        
        ## FOR DWT 1 REPRESENTATION
        #euclidean distance
        predClasses <- kNNCalculator(trainingActualClasses, dwt1EucMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "dwt", "euc", wSize=windowSize, kValue=k,
                                    dMax=NA, method="haar", sLength=NA, aSize=NA)
        solutionTable <- rbind(solutionTable, tempRow)
        #manhattan distance
        predClasses <- kNNCalculator(trainingActualClasses, dwt1ManhMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "dwt", "manh", wSize=windowSize, kValue=k,
                                    dMax=NA, method="haar", sLength=NA, aSize=NA)
        solutionTable <- rbind(solutionTable, tempRow)
        #dtw distance
        predClasses <- kNNCalculator(trainingActualClasses, dwt1DtwMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "dwt", "dtw", wSize=windowSize, kValue=k,
                                    dMax=NA, method="haar", sLength=NA, aSize=NA)
        solutionTable <- rbind(solutionTable, tempRow)
        #erp distance
        predClasses <- kNNCalculator(trainingActualClasses, dwt1ErpMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "dwt", "erp", wSize=windowSize, kValue=k,
                                    dMax=NA, method="haar", sLength=NA, aSize=NA)
        solutionTable <- rbind(solutionTable, tempRow)
        #lcss distance
        predClasses <- kNNCalculator(trainingActualClasses, dwt1LcssMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "dwt", "lcss", wSize=windowSize, kValue=k,
                                    dMax=NA, method="haar", sLength=NA, aSize=NA)
        solutionTable <- rbind(solutionTable, tempRow)
        
        ## FOR DWT 2 REPRESENTATION
        #euclidean distance
        predClasses <- kNNCalculator(trainingActualClasses, dwt2EucMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "dwt", "euc", wSize=windowSize, kValue=k,
                                    dMax=NA, method="la8", sLength=NA, aSize=NA)
        solutionTable <- rbind(solutionTable, tempRow)
        #manhattan distance
        predClasses <- kNNCalculator(trainingActualClasses, dwt2ManhMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "dwt", "manh", wSize=windowSize, kValue=k,
                                    dMax=NA, method="la8", sLength=NA, aSize=NA)
        solutionTable <- rbind(solutionTable, tempRow)
        #dtw distance
        predClasses <- kNNCalculator(trainingActualClasses, dwt2DtwMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "dwt", "dtw", wSize=windowSize, kValue=k,
                                    dMax=NA, method="la8", sLength=NA, aSize=NA)
        solutionTable <- rbind(solutionTable, tempRow)
        #erp distance
        predClasses <- kNNCalculator(trainingActualClasses, dwt2ErpMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "dwt", "erp", wSize=windowSize, kValue=k,
                                    dMax=NA, method="la8", sLength=NA, aSize=NA)
        solutionTable <- rbind(solutionTable, tempRow)
        #lcss distance
        predClasses <- kNNCalculator(trainingActualClasses, dwt2LcssMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "dwt", "lcss", wSize=windowSize, kValue=k,
                                    dMax=NA, method="la8", sLength=NA, aSize=NA)
        solutionTable <- rbind(solutionTable, tempRow)
        
        ## FOR SAX 1 REPRESENTATION
        #lcss distance
        predClasses <- kNNCalculator(trainingActualClasses, sax1LcssMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "sax", "lcss", wSize=windowSize, kValue=k,
                                    dMax=NA, method=NA, sLength=2, aSize=5)
        solutionTable <- rbind(solutionTable, tempRow)
    
        ## FOR SAX 2 REPRESENTATION
        #lcss distance
        predClasses <- kNNCalculator(trainingActualClasses, sax2LcssMatrix, k)
        tempRow <- accRowCalculator(testActualClasses, predClasses, "sax", "lcss", wSize=windowSize, kValue=k,
                                    dMax=NA, method=NA, sLength=4, aSize=8)
        solutionTable <- rbind(solutionTable, tempRow)
    }    
       
    solutionTable <- solutionTable[-1]
    return(solutionTable)
}

In [17]:
parameterTuning <- function(targetHour, windowSizeVec, kVec) {
    solutionTable <- data.table(repr=NA, dist=NA, wSize=NA, kNNValue=NA, 
                                maxDepth=NA, method=NA, segmentLength=NA, alphabetSize=NA, acc=NA)
    for (wSize in windowSizeVec) {
        dt_train <- getDataTableReady(dt[datetime<=training_period_end & datetime>=training_period_start], targetHour, wSize)
        dt_test <- getDataTableReady(dt[datetime<=testing_period_end & datetime>=testing_period_start], targetHour, wSize)
       
        tempModelTable <- modelComparisonFunc(dt_train, dt_test, kVec, wSize)
        solutionTable <- rbind(solutionTable, tempModelTable) 
    }
    solutionTable <- solutionTable[-1]
    return(solutionTable)    
}

In [18]:
predictTheFuture <- function(targetHour, windowSize, kNNValue, modelType, distType, maxDepth=NA, method=NA, 
                             sLength=NA, aSize=NA) {
    #preparing the sets
    currentTable <- getDataTableReady(dt, targetHour, windowSize)
    maxID <- as.numeric(currentTable[which.max(currentTable$id)]$id)
    cumulativeSerie <- currentTable[id!=maxID]
    predictedSerie <- currentTable[id==maxID]
    predictedSerie <- predictedSerie[, system_direction:=""]
    #actual classes of cumulativeSerie
    cumActualClasses <- actualClassesFinder(cumulativeSerie)
    
    #defining distance based functions
    distTypeEuc <- function(repr1, repr2) {
        distMatrix <- eucDistMatrix(repr1, rbind(repr2, repr2))
        KNN_neighborhood <- apply(distMatrix, 1, order)
        tempSplit <- split(cumActualClasses[KNN_neighborhood[1:kNNValue,]], 
                           ceiling(seq_along(cumActualClasses[KNN_neighborhood[1:kNNValue,]])/kNNValue))
        KNN_pred <- sapply(tempSplit, getmode)
        print(KNN_pred)
    }
    
    distTypeManh <- function(repr1, repr2) {
        distMatrix <- manhattanDistMatrix(repr1, rbind(repr2, repr2))
        KNN_neighborhood <- apply(distMatrix, 1, order)
        tempSplit <- split(cumActualClasses[KNN_neighborhood[1:kNNValue,]], 
                           ceiling(seq_along(cumActualClasses[KNN_neighborhood[1:kNNValue,]])/kNNValue))
        KNN_pred <- sapply(tempSplit, getmode)
        print(KNN_pred)
    }
    
    distTypeDtw <- function(repr1, repr2) {
        distMatrix <- dtwDistMatrix(repr1, repr2)
        KNN_neighborhood <- apply(distMatrix, 2, order)
        tempSplit <- split(cumActualClasses[KNN_neighborhood[1:kNNValue,]], 
                           ceiling(seq_along(cumActualClasses[KNN_neighborhood[1:kNNValue,]])/kNNValue))
        KNN_pred <- sapply(tempSplit, getmode)
        print(KNN_pred) 
    }
    
    distTypeErp <- function(repr1, repr2) {
        distMatrix <- erpDistMatrix(repr1, rbind(repr2, repr2))
        KNN_neighborhood <- apply(distMatrix, 1, order)
        tempSplit <- split(cumActualClasses[KNN_neighborhood[1:kNNValue,]], 
                           ceiling(seq_along(cumActualClasses[KNN_neighborhood[1:kNNValue,]])/kNNValue))
        KNN_pred <- sapply(tempSplit, getmode)
        print(KNN_pred)
    }
    
    distTypeLcssNumeric <- function(repr1, repr2) {
        distMatrix <- lcssDistMatrixNumeric(repr1, rbind(repr2, repr2))
        KNN_neighborhood <- apply(distMatrix, 1, order)
        tempSplit <- split(cumActualClasses[KNN_neighborhood[1:kNNValue,]], 
                           ceiling(seq_along(cumActualClasses[KNN_neighborhood[1:kNNValue,]])/kNNValue))
        KNN_pred <- sapply(tempSplit, getmode)
        print(KNN_pred)
    }
    
    distTypeLcssString <- function(repr1, repr2) {
        distMatrix <- lcssDistMatrixString(repr1, repr2)
        KNN_neighborhood <- apply(distMatrix, 1, order)
        tempSplit <- split(cumActualClasses[KNN_neighborhood[1:kNNValue,]], 
                           ceiling(seq_along(cumActualClasses[KNN_neighborhood[1:kNNValue,]])/kNNValue))
        KNN_pred <- sapply(tempSplit, getmode)
        print(KNN_pred) 
    }
    
    #FOR TREE MODEL
    if (modelType=="tree") {
        #for repr1
        repr1 <- treeReprFunc(cumulativeSerie, maxDepth)
        #for repr2
        treeFit <- rpart(net~datetime, predictedSerie[1:windowSize],
                         control=rpart.control(maxdepth=maxDepth, cp=0, minsplit=3, minbucket=1))
        repr2 <- predict(treeFit, predictedSerie[1:windowSize])     
        repr2 <- matrix(repr2, nrow=1, ncol=windowSize, byrow=T)
        
        #distance selection
        if (distType=="euc") {  
            distTypeEuc(repr1, repr2)
        }
        else if (distType=="manh") {
            distTypeManh(repr1, repr2)
        }
        else if(distType=="dtw") {
            distTypeDtw(repr1, repr2)
        } 
        else if(distType=="erp") {
            distTypeErp(repr1, repr2)
        }
        else if(distType=="lcss") {
            distTypeLcssNumeric(repr1, repr2)
        }
    }
    
    #FOR DWT MODEL
    if (modelType=="dwt") {
        #for repr1
        repr1 <- waveletReprFunc(cumulativeSerie, method)
        #for repr2
        dwtObject <- dwt(predictedSerie$net[1:windowSize], method)   
        repr2 <- dwtObject@V[[1]] 
        repr2 <- matrix(repr2, nrow=1, ncol=windowSize/2, byrow=T)
        
        #distance selection
        if (distType=="euc") {  
            distTypeEuc(repr1, repr2)
        }
        else if (distType=="manh") {
            distTypeManh(repr1, repr2)
        }
        else if(distType=="dtw") {
            distTypeDtw(repr1, repr2)
        } 
        else if(distType=="erp") {
            distTypeErp(repr1, repr2)
        }
        else if(distType=="lcss") {
            distTypeLcssNumeric(repr1, repr2)
        }
    }
    
    #FOR SAX MODEL        
    if (modelType=="sax") {
        #for repr1
        repr1 <- saxReprFunc(cumulativeSerie, sLength, aSize)
        #for repr2
        repr2 <- repr_sax(predictedSerie$net[1:windowSize], q=sLength, a=aSize)
        repr2 <- paste(repr2, collapse="")
        
        #only lcss distance is defined
        distTypeLcssString(repr1, repr2)
    }
}

# Classification Domain Model Trials

In [17]:
#the last 7 days are testing period
testing_period_start <- as.POSIXct("2022-01-02 00:00:00")
testing_period_end <- as.POSIXct("2022-01-08 23:00:00")

training_period_start <- as.POSIXct("2019-01-01 23:00:00")
training_period_end <- as.POSIXct("2022-01-01 23:00:00")

### FOR 12.00

In [18]:
start_time <- Sys.time()
targetHour <- 12
windowSize <- c(8, 16, 24)
kVector <- c(1, 3, 5)

table12 <- parameterTuning(targetHour, windowSize, kVector)
table12[order(acc, decreasing=T)]
end_time <- Sys.time()

repr,dist,wSize,kNNValue,maxDepth,method,segmentLength,alphabetSize,acc
raw,dtw,16,3,NA,NA,NA,NA,1.0000000
tree,dtw,16,3,6,NA,NA,NA,1.0000000
dwt,dtw,16,3,NA,haar,NA,NA,1.0000000
raw,dtw,16,5,NA,NA,NA,NA,1.0000000
raw,erp,16,5,NA,NA,NA,NA,1.0000000
tree,dtw,16,5,6,NA,NA,NA,1.0000000
dwt,dtw,16,5,NA,haar,NA,NA,1.0000000
tree,dtw,24,5,2,NA,NA,NA,1.0000000
sax,lcss,8,1,NA,NA,2,5,0.8333333
sax,lcss,8,1,NA,NA,4,8,0.8333333


In [19]:
head(table12[order(acc, decreasing=T)], 8)

repr,dist,wSize,kNNValue,maxDepth,method,segmentLength,alphabetSize,acc
raw,dtw,16,3,NA,NA,NA,NA,1
tree,dtw,16,3,6,NA,NA,NA,1
dwt,dtw,16,3,NA,haar,NA,NA,1
raw,dtw,16,5,NA,NA,NA,NA,1
raw,erp,16,5,NA,NA,NA,NA,1
tree,dtw,16,5,6,NA,NA,NA,1
dwt,dtw,16,5,NA,haar,NA,NA,1
tree,dtw,24,5,2,NA,NA,NA,1


In [20]:
end_time - start_time

Time difference of 6.779962 mins

### FOR 13.00

In [21]:
start_time <- Sys.time()
targetHour <- 13
windowSize <- c(8, 16, 24)
kVector <- c(1, 3, 5)

table13 <- parameterTuning(targetHour, windowSize, kVector)
table13[order(acc, decreasing=T)]
end_time <- Sys.time()

repr,dist,wSize,kNNValue,maxDepth,method,segmentLength,alphabetSize,acc
tree,euc,8,5,2,NA,NA,NA,0.8333333
sax,lcss,8,5,NA,NA,2,5,0.8333333
dwt,erp,16,3,NA,haar,NA,NA,0.8333333
raw,dtw,16,5,NA,NA,NA,NA,0.8333333
raw,erp,16,5,NA,NA,NA,NA,0.8333333
tree,erp,16,5,6,NA,NA,NA,0.8333333
dwt,manh,24,1,NA,la8,NA,NA,0.8333333
dwt,erp,24,1,NA,la8,NA,NA,0.8333333
tree,dtw,24,5,2,NA,NA,NA,0.8333333
dwt,erp,24,5,NA,haar,NA,NA,0.8333333


In [22]:
head(table13[order(acc, decreasing=T)], 8)

repr,dist,wSize,kNNValue,maxDepth,method,segmentLength,alphabetSize,acc
tree,euc,8,5,2,NA,NA,NA,0.8333333
sax,lcss,8,5,NA,NA,2,5,0.8333333
dwt,erp,16,3,NA,haar,NA,NA,0.8333333
raw,dtw,16,5,NA,NA,NA,NA,0.8333333
raw,erp,16,5,NA,NA,NA,NA,0.8333333
tree,erp,16,5,6,NA,NA,NA,0.8333333
dwt,manh,24,1,NA,la8,NA,NA,0.8333333
dwt,erp,24,1,NA,la8,NA,NA,0.8333333


In [23]:
end_time - start_time

Time difference of 6.282409 mins

### FOR 14:00

In [24]:
start_time <- Sys.time()
targetHour <- 14
windowSize <- c(8, 16, 24)
kVector <- c(1, 3, 5)

table14 <- parameterTuning(targetHour, windowSize, kVector)
table14[order(acc, decreasing=T)]
end_time <- Sys.time()

repr,dist,wSize,kNNValue,maxDepth,method,segmentLength,alphabetSize,acc
sax,lcss,8,5,NA,NA,2,5,1.0000000
raw,euc,16,1,NA,NA,NA,NA,1.0000000
raw,dtw,16,1,NA,NA,NA,NA,1.0000000
tree,dtw,16,1,6,NA,NA,NA,1.0000000
raw,euc,16,3,NA,NA,NA,NA,1.0000000
raw,dtw,16,3,NA,NA,NA,NA,1.0000000
tree,euc,16,3,2,NA,NA,NA,1.0000000
tree,manh,16,3,2,NA,NA,NA,1.0000000
tree,dtw,16,3,6,NA,NA,NA,1.0000000
dwt,erp,16,3,NA,la8,NA,NA,1.0000000


In [25]:
head(table14[order(acc, decreasing=T)], 8)

repr,dist,wSize,kNNValue,maxDepth,method,segmentLength,alphabetSize,acc
sax,lcss,8,5,NA,NA,2,5,1
raw,euc,16,1,NA,NA,NA,NA,1
raw,dtw,16,1,NA,NA,NA,NA,1
tree,dtw,16,1,6,NA,NA,NA,1
raw,euc,16,3,NA,NA,NA,NA,1
raw,dtw,16,3,NA,NA,NA,NA,1
tree,euc,16,3,2,NA,NA,NA,1
tree,manh,16,3,2,NA,NA,NA,1


In [26]:
end_time - start_time

Time difference of 6.292017 mins

### FOR 15:00

In [27]:
start_time <- Sys.time()
targetHour <- 15
windowSize <- c(8, 16, 24)
kVector <- c(1, 3, 5)

table15 <- parameterTuning(targetHour, windowSize, kVector)
table15[order(acc, decreasing=T)]
end_time <- Sys.time()

repr,dist,wSize,kNNValue,maxDepth,method,segmentLength,alphabetSize,acc
tree,euc,8,5,2,NA,NA,NA,1.0000000
dwt,dtw,8,5,NA,haar,NA,NA,1.0000000
sax,lcss,8,5,NA,NA,2,5,1.0000000
raw,euc,16,1,NA,NA,NA,NA,1.0000000
dwt,manh,16,1,NA,haar,NA,NA,1.0000000
tree,manh,16,3,2,NA,NA,NA,1.0000000
dwt,manh,16,3,NA,haar,NA,NA,1.0000000
dwt,dtw,16,3,NA,haar,NA,NA,1.0000000
dwt,erp,16,3,NA,la8,NA,NA,1.0000000
raw,dtw,16,5,NA,NA,NA,NA,1.0000000


In [28]:
head(table15[order(acc, decreasing=T)], 8)

repr,dist,wSize,kNNValue,maxDepth,method,segmentLength,alphabetSize,acc
tree,euc,8,5,2,NA,NA,NA,1
dwt,dtw,8,5,NA,haar,NA,NA,1
sax,lcss,8,5,NA,NA,2,5,1
raw,euc,16,1,NA,NA,NA,NA,1
dwt,manh,16,1,NA,haar,NA,NA,1
tree,manh,16,3,2,NA,NA,NA,1
dwt,manh,16,3,NA,haar,NA,NA,1
dwt,dtw,16,3,NA,haar,NA,NA,1


In [29]:
end_time - start_time

Time difference of 6.262318 mins

### FOR 16:00

In [30]:
start_time <- Sys.time()
targetHour <- 16
windowSize <- c(8, 16, 24)
kVector <- c(1, 3, 5)

table16 <- parameterTuning(targetHour, windowSize, kVector)
table16[order(acc, decreasing=T)]
end_time <- Sys.time()

repr,dist,wSize,kNNValue,maxDepth,method,segmentLength,alphabetSize,acc
dwt,dtw,8,1,NA,haar,NA,NA,1
dwt,dtw,8,3,NA,haar,NA,NA,1
dwt,euc,8,3,NA,la8,NA,NA,1
dwt,dtw,8,5,NA,haar,NA,NA,1
dwt,euc,8,5,NA,la8,NA,NA,1
raw,euc,16,1,NA,NA,NA,NA,1
raw,manh,16,1,NA,NA,NA,NA,1
tree,euc,16,1,2,NA,NA,NA,1
tree,euc,16,1,6,NA,NA,NA,1
tree,Manh,16,1,6,NA,NA,NA,1


In [31]:
head(table16[order(acc, decreasing=T)], 8)

repr,dist,wSize,kNNValue,maxDepth,method,segmentLength,alphabetSize,acc
dwt,dtw,8,1,NA,haar,NA,NA,1
dwt,dtw,8,3,NA,haar,NA,NA,1
dwt,euc,8,3,NA,la8,NA,NA,1
dwt,dtw,8,5,NA,haar,NA,NA,1
dwt,euc,8,5,NA,la8,NA,NA,1
raw,euc,16,1,NA,NA,NA,NA,1
raw,manh,16,1,NA,NA,NA,NA,1
tree,euc,16,1,2,NA,NA,NA,1


In [32]:
end_time - start_time

Time difference of 6.342153 mins

### FOR 17:00

In [33]:
start_time <- Sys.time()
targetHour <- 17
windowSize <- c(8, 16, 24)
kVector <- c(1, 3, 5)

table17 <- parameterTuning(targetHour, windowSize, kVector)
table17[order(acc, decreasing=T)]
end_time <- Sys.time()

repr,dist,wSize,kNNValue,maxDepth,method,segmentLength,alphabetSize,acc
tree,dtw,8,3,2,NA,NA,NA,1.0000000
tree,dtw,8,5,2,NA,NA,NA,1.0000000
dwt,dtw,16,1,NA,haar,NA,NA,1.0000000
dwt,dtw,16,3,NA,haar,NA,NA,1.0000000
dwt,dtw,16,5,NA,haar,NA,NA,1.0000000
raw,manh,24,1,NA,NA,NA,NA,1.0000000
tree,Manh,24,1,6,NA,NA,NA,1.0000000
dwt,euc,24,1,NA,haar,NA,NA,1.0000000
tree,erp,24,3,6,NA,NA,NA,1.0000000
dwt,erp,24,3,NA,haar,NA,NA,1.0000000


In [34]:
head(table17[order(acc, decreasing=T)], 8)

repr,dist,wSize,kNNValue,maxDepth,method,segmentLength,alphabetSize,acc
tree,dtw,8,3,2,NA,NA,NA,1
tree,dtw,8,5,2,NA,NA,NA,1
dwt,dtw,16,1,NA,haar,NA,NA,1
dwt,dtw,16,3,NA,haar,NA,NA,1
dwt,dtw,16,5,NA,haar,NA,NA,1
raw,manh,24,1,NA,NA,NA,NA,1
tree,Manh,24,1,6,NA,NA,NA,1
dwt,euc,24,1,NA,haar,NA,NA,1


In [35]:
end_time - start_time

Time difference of 7.203929 mins

### FOR 18:00

In [36]:
start_time <- Sys.time()
targetHour <- 18
windowSize <- c(8, 16, 24)
kVector <- c(1, 3, 5)

table18 <- parameterTuning(targetHour, windowSize, kVector)
table18[order(acc, decreasing=T)]
end_time <- Sys.time()

repr,dist,wSize,kNNValue,maxDepth,method,segmentLength,alphabetSize,acc
tree,dtw,24,1,6,NA,NA,NA,1.0000000
dwt,dtw,24,5,NA,la8,NA,NA,1.0000000
raw,manh,8,1,NA,NA,NA,NA,0.8333333
raw,erp,8,1,NA,NA,NA,NA,0.8333333
tree,euc,8,1,2,NA,NA,NA,0.8333333
tree,erp,8,1,2,NA,NA,NA,0.8333333
tree,euc,8,1,6,NA,NA,NA,0.8333333
tree,erp,8,1,6,NA,NA,NA,0.8333333
dwt,euc,8,1,NA,haar,NA,NA,0.8333333
dwt,manh,8,1,NA,haar,NA,NA,0.8333333


In [37]:
head(table18[order(acc, decreasing=T)], 8)

repr,dist,wSize,kNNValue,maxDepth,method,segmentLength,alphabetSize,acc
tree,dtw,24,1,6,NA,NA,NA,1.0000000
dwt,dtw,24,5,NA,la8,NA,NA,1.0000000
raw,manh,8,1,NA,NA,NA,NA,0.8333333
raw,erp,8,1,NA,NA,NA,NA,0.8333333
tree,euc,8,1,2,NA,NA,NA,0.8333333
tree,erp,8,1,2,NA,NA,NA,0.8333333
tree,euc,8,1,6,NA,NA,NA,0.8333333
tree,erp,8,1,6,NA,NA,NA,0.8333333


In [38]:
end_time - start_time

Time difference of 6.895299 mins

### FOR 19:00

In [39]:
start_time <- Sys.time()
targetHour <- 19
windowSize <- c(8, 16, 24)
kVector <- c(1, 3, 5)

table19 <- parameterTuning(targetHour, windowSize, kVector)
table19[order(acc, decreasing=T)]
end_time <- Sys.time()

repr,dist,wSize,kNNValue,maxDepth,method,segmentLength,alphabetSize,acc
raw,erp,8,3,NA,NA,NA,NA,1.0000000
tree,dtw,8,1,2,NA,NA,NA,0.8333333
dwt,euc,8,1,NA,la8,NA,NA,0.8333333
dwt,manh,8,1,NA,la8,NA,NA,0.8333333
dwt,dtw,8,1,NA,la8,NA,NA,0.8333333
dwt,erp,8,1,NA,la8,NA,NA,0.8333333
sax,lcss,8,1,NA,NA,2,5,0.8333333
sax,lcss,8,1,NA,NA,4,8,0.8333333
raw,euc,8,3,NA,NA,NA,NA,0.8333333
raw,manh,8,3,NA,NA,NA,NA,0.8333333


In [40]:
head(table19[order(acc, decreasing=T)], 8)

repr,dist,wSize,kNNValue,maxDepth,method,segmentLength,alphabetSize,acc
raw,erp,8,3,NA,NA,NA,NA,1.0000000
tree,dtw,8,1,2,NA,NA,NA,0.8333333
dwt,euc,8,1,NA,la8,NA,NA,0.8333333
dwt,manh,8,1,NA,la8,NA,NA,0.8333333
dwt,dtw,8,1,NA,la8,NA,NA,0.8333333
dwt,erp,8,1,NA,la8,NA,NA,0.8333333
sax,lcss,8,1,NA,NA,2,5,0.8333333
sax,lcss,8,1,NA,NA,4,8,0.8333333


In [41]:
end_time - start_time

Time difference of 6.272239 mins

### FOR 20:00

In [42]:
start_time <- Sys.time()
targetHour <- 20
windowSize <- c(8, 16, 24)
kVector <- c(1, 3, 5)

table20 <- parameterTuning(targetHour, windowSize, kVector)
table20[order(acc, decreasing=T)]
end_time <- Sys.time()

repr,dist,wSize,kNNValue,maxDepth,method,segmentLength,alphabetSize,acc
raw,euc,8,3,NA,NA,NA,NA,1.0000000
raw,euc,8,5,NA,NA,NA,NA,1.0000000
tree,euc,8,5,6,NA,NA,NA,1.0000000
tree,Manh,8,5,6,NA,NA,NA,1.0000000
tree,erp,8,5,6,NA,NA,NA,1.0000000
tree,dtw,8,1,2,NA,NA,NA,0.8333333
dwt,manh,8,1,NA,la8,NA,NA,0.8333333
dwt,dtw,8,1,NA,la8,NA,NA,0.8333333
dwt,erp,8,1,NA,la8,NA,NA,0.8333333
sax,lcss,8,1,NA,NA,2,5,0.8333333


In [43]:
head(table20[order(acc, decreasing=T)], 8)

repr,dist,wSize,kNNValue,maxDepth,method,segmentLength,alphabetSize,acc
raw,euc,8,3,NA,NA,NA,NA,1.0000000
raw,euc,8,5,NA,NA,NA,NA,1.0000000
tree,euc,8,5,6,NA,NA,NA,1.0000000
tree,Manh,8,5,6,NA,NA,NA,1.0000000
tree,erp,8,5,6,NA,NA,NA,1.0000000
tree,dtw,8,1,2,NA,NA,NA,0.8333333
dwt,manh,8,1,NA,la8,NA,NA,0.8333333
dwt,dtw,8,1,NA,la8,NA,NA,0.8333333


In [44]:
end_time - start_time

Time difference of 6.268416 mins

### FOR 21:00

In [45]:
start_time <- Sys.time()
targetHour <- 21
windowSize <- c(8, 16, 24)
kVector <- c(1, 3, 5)

table21 <- parameterTuning(targetHour, windowSize, kVector)
table21[order(acc, decreasing=T)]
end_time <- Sys.time()

repr,dist,wSize,kNNValue,maxDepth,method,segmentLength,alphabetSize,acc
raw,euc,8,1,NA,NA,NA,NA,1.0000000
dwt,manh,8,1,NA,la8,NA,NA,1.0000000
dwt,dtw,8,1,NA,la8,NA,NA,1.0000000
dwt,erp,8,1,NA,la8,NA,NA,1.0000000
raw,euc,8,3,NA,NA,NA,NA,1.0000000
raw,erp,8,3,NA,NA,NA,NA,1.0000000
dwt,dtw,8,3,NA,la8,NA,NA,1.0000000
raw,euc,8,5,NA,NA,NA,NA,1.0000000
tree,euc,8,5,6,NA,NA,NA,1.0000000
dwt,dtw,8,5,NA,la8,NA,NA,1.0000000


In [46]:
head(table21[order(acc, decreasing=T)], 8)

repr,dist,wSize,kNNValue,maxDepth,method,segmentLength,alphabetSize,acc
raw,euc,8,1,NA,NA,NA,NA,1
dwt,manh,8,1,NA,la8,NA,NA,1
dwt,dtw,8,1,NA,la8,NA,NA,1
dwt,erp,8,1,NA,la8,NA,NA,1
raw,euc,8,3,NA,NA,NA,NA,1
raw,erp,8,3,NA,NA,NA,NA,1
dwt,dtw,8,3,NA,la8,NA,NA,1
raw,euc,8,5,NA,NA,NA,NA,1


In [47]:
end_time - start_time

Time difference of 6.280159 mins

### FOR 22:00

In [48]:
start_time <- Sys.time()
targetHour <- 22
windowSize <- c(8, 16, 24)
kVector <- c(1, 3, 5)

table22 <- parameterTuning(targetHour, windowSize, kVector)
table22[order(acc, decreasing=T)]
end_time <- Sys.time()

repr,dist,wSize,kNNValue,maxDepth,method,segmentLength,alphabetSize,acc
raw,euc,8,1,NA,NA,NA,NA,0.8333333
raw,manh,8,1,NA,NA,NA,NA,0.8333333
dwt,euc,8,1,NA,haar,NA,NA,0.8333333
dwt,erp,8,3,NA,la8,NA,NA,0.8333333
raw,manh,8,5,NA,NA,NA,NA,0.8333333
raw,erp,8,5,NA,NA,NA,NA,0.8333333
tree,manh,8,5,2,NA,NA,NA,0.8333333
tree,Manh,8,5,6,NA,NA,NA,0.8333333
dwt,dtw,8,5,NA,haar,NA,NA,0.8333333
tree,euc,16,1,2,NA,NA,NA,0.8333333


In [49]:
head(table22[order(acc, decreasing=T)], 8)

repr,dist,wSize,kNNValue,maxDepth,method,segmentLength,alphabetSize,acc
raw,euc,8,1,NA,NA,NA,NA,0.8333333
raw,manh,8,1,NA,NA,NA,NA,0.8333333
dwt,euc,8,1,NA,haar,NA,NA,0.8333333
dwt,erp,8,3,NA,la8,NA,NA,0.8333333
raw,manh,8,5,NA,NA,NA,NA,0.8333333
raw,erp,8,5,NA,NA,NA,NA,0.8333333
tree,manh,8,5,2,NA,NA,NA,0.8333333
tree,Manh,8,5,6,NA,NA,NA,0.8333333


In [50]:
end_time - start_time

Time difference of 6.281275 mins

### FOR 23:00

In [51]:
start_time <- Sys.time()
targetHour <- 23
windowSize <- c(8, 16, 24)
kVector <- c(1, 3, 5)

table23 <- parameterTuning(targetHour, windowSize, kVector)
table23[order(acc, decreasing=T)]
end_time <- Sys.time()

repr,dist,wSize,kNNValue,maxDepth,method,segmentLength,alphabetSize,acc
raw,manh,8,1,NA,NA,NA,NA,0.8333333
raw,dtw,8,1,NA,NA,NA,NA,0.8333333
tree,dtw,8,1,2,NA,NA,NA,0.8333333
tree,dtw,8,1,6,NA,NA,NA,0.8333333
tree,erp,8,1,6,NA,NA,NA,0.8333333
dwt,euc,8,1,NA,haar,NA,NA,0.8333333
dwt,manh,8,1,NA,haar,NA,NA,0.8333333
dwt,erp,8,1,NA,haar,NA,NA,0.8333333
sax,lcss,8,1,NA,NA,2,5,0.8333333
raw,manh,8,3,NA,NA,NA,NA,0.8333333


In [52]:
head(table23[order(acc, decreasing=T)], 8)

repr,dist,wSize,kNNValue,maxDepth,method,segmentLength,alphabetSize,acc
raw,manh,8,1,NA,NA,NA,NA,0.8333333
raw,dtw,8,1,NA,NA,NA,NA,0.8333333
tree,dtw,8,1,2,NA,NA,NA,0.8333333
tree,dtw,8,1,6,NA,NA,NA,0.8333333
tree,erp,8,1,6,NA,NA,NA,0.8333333
dwt,euc,8,1,NA,haar,NA,NA,0.8333333
dwt,manh,8,1,NA,haar,NA,NA,0.8333333
dwt,erp,8,1,NA,haar,NA,NA,0.8333333


In [53]:
end_time - start_time

Time difference of 6.273798 mins

# Predicting by modelSet1

### For 12:00

In [24]:
predictTheFuture(12, 16, 5, modelType="dwt", distType="dtw", method="haar")

         1 
"Positive" 


### For 13:00

In [28]:
predictTheFuture(13, 8, 5, modelType="tree", distType="euc", maxDepth=2)

         1          2 
"Positive" "Positive" 


### For 14:00

In [32]:
predictTheFuture(14, 8, 5, modelType="sax", distType="lcss", sLength=2, aSize=5)

         1 
"Negative" 


### For 15:00

In [36]:
predictTheFuture(15, 8, 5, modelType="dwt", distType="dtw", method="haar")

         1 
"Positive" 


### For 16:00

In [40]:
predictTheFuture(16, 8, 5, modelType="dwt", distType="dtw", method="haar")

         1 
"Positive" 


### For 17:00

In [44]:
predictTheFuture(17, 8, 3, modelType="tree", distType="dtw", maxDepth=2)

         1 
"Positive" 


### For 18:00

In [48]:
predictTheFuture(18, 24, 5, modelType="dwt", distType="dtw", method="la8")

         1 
"Positive" 


### For 19:00

In [52]:
predictTheFuture(19, 16, 1, modelType="sax", distType="lcss", sLength=2, aSize=5)

         1 
"Negative" 


### For 20:00

In [56]:
predictTheFuture(20, 8, 5, modelType="tree", distType="euc", maxDepth=6)

         1          2 
"Positive" "Positive" 


### For 21:00

In [60]:
predictTheFuture(21, 8, 1, modelType="dwt", distType="dtw", method="la8")

         1 
"Positive" 


### For 22:00

In [64]:
predictTheFuture(22, 8, 1, modelType="dwt", distType="euc", method="haar")

         1          2 
"Positive" "Positive" 


### For 23:00

In [68]:
predictTheFuture(23, 8, 1, modelType="tree", distType="dtw", maxDepth=2)

         1 
"Positive" 


# Predicting by modelSet2

In [69]:
# FOR 12:00
predictTheFuture(12, 16, 3, modelType="tree", distType="dtw", maxDepth=6)
predictTheFuture(12, 16, 3, modelType="dwt", distType="dtw", method="haar")
predictTheFuture(12, 24, 5, modelType="tree", distType="dtw", maxDepth=2)
#print("additional")
#predictTheFuture(12, 8, 5, modelType="sax", distType="lcss", sLength=2, aSize=5)

         1 
"Positive" 
         1 
"Positive" 
         1 
"Positive" 


In [70]:
# FOR 13:00
predictTheFuture(13, 8, 5, modelType="tree", distType="euc", maxDepth=2)
predictTheFuture(13, 8, 5, modelType="sax", distType="lcss", sLength=2, aSize=5)
predictTheFuture(13, 16, 3, modelType="dwt", distType="erp", method="haar")
#print("additional")
#predictTheFuture(13, 8, 5, modelType="sax", distType="lcss", sLength=2, aSize=5)

         1          2 
"Positive" "Positive" 
         1 
"Positive" 
         1          2 
"Positive" "Positive" 


In [71]:
# FOR 14:00
predictTheFuture(14, 16, 3, modelType="tree", distType="euc", maxDepth=2)
predictTheFuture(14, 16, 1, modelType="tree", distType="dtw", maxDepth=6)
predictTheFuture(14, 8, 5, modelType="sax", distType="lcss", sLength=2, aSize=5)

         1          2 
"Negative" "Negative" 
         1 
"Positive" 
         1 
"Negative" 


In [72]:
# FOR 15:00
predictTheFuture(15, 8, 5, modelType="tree", distType="euc", maxDepth=2)
predictTheFuture(15, 8, 5, modelType="dwt", distType="dtw", method="haar")
predictTheFuture(15, 8, 5, modelType="sax", distType="lcss", sLength=2, aSize=5)

         1          2 
"Positive" "Positive" 
         1 
"Positive" 
         1 
"Negative" 


In [73]:
# FOR 16:00
predictTheFuture(16, 8, 5, modelType="dwt", distType="dtw", method="haar")
predictTheFuture(16, 8, 3, modelType="dwt", distType="euc", method="la8")
predictTheFuture(16, 16, 1, modelType="tree", distType="euc", maxDepth=2)

         1 
"Positive" 
         1          2 
"Positive" "Positive" 
         1          2 
"Negative" "Negative" 


In [74]:
# FOR 17:00
predictTheFuture(17, 8, 3, modelType="tree", distType="dtw", maxDepth=2)
predictTheFuture(17, 24, 1, modelType="tree", distType="manh", maxDepth=6)
predictTheFuture(17, 16, 3, modelType="dwt", distType="dtw", method="haar")

         1 
"Positive" 
         1          2 
"Positive" "Positive" 
         1 
"Positive" 


In [75]:
# FOR 18:00
predictTheFuture(18, 8, 1, modelType="dwt", distType="dtw", method="haar")
predictTheFuture(18, 24, 1, modelType="tree", distType="dtw", maxDepth=6)
predictTheFuture(18, 24, 5, modelType="dwt", distType="dtw", method="la8")

         1 
"Positive" 
         1 
"Negative" 
         1 
"Positive" 


In [76]:
# FOR 19:00
predictTheFuture(19, 8, 1, modelType="tree", distType="dtw", maxDepth=2)
predictTheFuture(19, 8, 1, modelType="dwt", distType="euc", method="la8")
predictTheFuture(19, 8, 1, modelType="sax", distType="lcss", sLength=4, aSize=8)

         1 
"Positive" 
         1          2 
"Negative" "Negative" 
         1 
"Negative" 


In [77]:
# FOR 20:00
predictTheFuture(20, 8, 5, modelType="tree", distType="euc", maxDepth=6)
predictTheFuture(20, 8, 5, modelType="tree", distType="manh", maxDepth=6)
predictTheFuture(20, 8, 5, modelType="tree", distType="erp", maxDepth=6)

         1          2 
"Positive" "Positive" 
         1          2 
"Positive" "Positive" 
         1          2 
"Positive" "Positive" 


In [78]:
# FOR 21:00
predictTheFuture(21, 16, 5, modelType="tree", distType="dtw", maxDepth=2)
predictTheFuture(21, 16, 3, modelType="tree", distType="erp", maxDepth=6)
predictTheFuture(21, 8, 5, modelType="dwt", distType="dtw", method="la8")

         1 
"Negative" 
         1          2 
"Positive" "Positive" 
         1 
"Positive" 


In [79]:
# FOR 22:00
predictTheFuture(22, 8, 5, modelType="tree", distType="manh", maxDepth=2)
predictTheFuture(22, 16, 5, modelType="tree", distType="erp", maxDepth=2)
predictTheFuture(22, 8, 3, modelType="dwt", distType="erp", method="la8")

         1          2 
"Positive" "Positive" 
         1          2 
"Positive" "Positive" 
         1          2 
"Positive" "Positive" 


In [80]:
# FOR 23:00
predictTheFuture(23, 16, 3, modelType="tree", distType="euc", maxDepth=2)
predictTheFuture(23, 8, 5, modelType="dwt", distType="erp", method="la8")
predictTheFuture(23, 8, 3, modelType="sax", distType="lcss", sLength=4, aSize=8)

         1          2 
"Negative" "Negative" 
        1         2 
"Neutral" "Neutral" 
         1 
"Negative" 
